In [9]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
import os
from dotenv import load_dotenv

from pydantic import BaseModel, Field

from google.adk import Agent
from google.adk.tools import google_search 

from google.adk.runners import InMemoryRunner
from google.genai import types

import google.generativeai as genai
from google.generativeai.types import ContentType
from PIL import Image
from IPython.display import Markdown

from crewai_tools import FileWriterTool

# LangChain tools
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from google.adk.tools.langchain_tool import LangchainTool


In [13]:
load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")
model_name = os.getenv("MODEL") or "models/gemini-2.5-pro"

genai.configure(api_key=api_key)

In [12]:

for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
    	print(m.name)

models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image


In [15]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')

In [16]:
class StockAnalysis(BaseModel):
    ticker: str = Field(description="Stock ticker symbol")
    company_name: str = Field(description="Full company name if known")
    sector: str = Field(description="Business sector")    
    summary: str = Field(description="Short explanation of the stock")


In [18]:
from langchain_community.utilities import WikipediaAPIWrapper

wiki_api = WikipediaAPIWrapper(lang="en", top_k_results=1)

def wikipedia_search(query: str):
    return wiki_api.run(query)

def write_file(path: str, content: str):
    """Write text to a file."""
    with open(path, "w", encoding="utf-8") as f:
        f.write(content)
    return f"Saved file: {path}"


In [19]:
research_agent = Agent(
    model=model_name,
    name="research_agent",
    instruction=(
        "You are a stock research assistant. "
        "You can search Wikipedia (via wikipedia_search) and write files (via write_file)."
    ),
    tools=[wikipedia_search, write_file],  # only plain functions
    output_schema=StockAnalysis
)

google_agent = Agent(
    model=model_name,
    name="google_agent",
    instruction="You are a Google search assistant. Use Google Search only.",
    tools=[google_search]  # pass as-is
)

In [ ]:
from google.adk.runners import InMemoryRunner

user_id = "user_app"
app_name="stock_app"

runner = InMemoryRunner(
    agent=research_agent,
    app_name=app_name
)

session = await runner.session_service.create_session(
    app_name=app_name,
    user_id=user_id
)

print("Session wikipedia created:", session.id)

Session created: 41e3b124-e481-4e18-a694-e742dd532bce


In [36]:
runner_google = InMemoryRunner(agent=google_agent, app_name="google_app")

session_google = await runner_google.session_service.create_session(
    app_name="google_app",
    user_id=user_id
)

print("Session Google created:", session_google.id)

Session Google created: f6f9e17c-f1c9-41c4-ae15-f09ee371155f


In [37]:
from google.genai import types

async def ask_agent(question: str, use_google=False):
    print("\nUSER:", question)
    
    google_result = ""
    if use_google:
        content = types.Content(role="user", parts=[{"text": question}])
        async for event in runner_google.run_async(
            user_id=user_id,
            session_id=session_google.id,
            new_message=content
        ):
            for part in event.content.parts:
                if hasattr(part, "text") and part.text:
                    google_result = part.text
                    print("Google →", google_result)
    
    # Send question + Google result to research_agent
    final_prompt = question
    if google_result:
        final_prompt += f"\n\nGoogle info: {google_result}"
    
    content = types.Content(role="user", parts=[{"text": final_prompt}])
    async for event in runner.run_async(
        user_id=user_id,
        session_id=session.id,
        new_message=content
    ):
        for part in event.content.parts:
            if hasattr(part, "text") and part.text:
                print("research_agent →", part.text)


In [38]:
# runner
import asyncio

async def main(question: str):
    await ask_agent(question)
 


In [ ]:

asyncio.run(main("Summarize Tesla's business model."))


USER: Summarize Tesla's business model from Wikipedia.
research_agent → {"ticker": "TSLA", "company_name": "Tesla, Inc.", "sector": "Automotive and Clean Energy", "summary": "Tesla, Inc. designs, manufactures, and sells battery electric vehicles (BEVs), stationary battery energy storage devices for home to grid-scale, solar panels and solar shingles, and related products and services."}


In [39]:

await ask_agent("Give me the latest stock news about AAPL.", use_google=True)


USER: Give me the latest stock news about AAPL.
Google → Here's the latest stock news regarding Apple (AAPL):

**Current Stock Performance & Analyst Outlook:**
As of Thursday, November 28, 2025, Apple (AAPL) shares are trading around $277.55, showing a slight increase of 0.21% in the past 24 hours. The stock has seen a 4.53% rise over the past week and a 4.78% increase in the last month. Over the last year, Apple Inc. has experienced an 18.95% increase. Analysts have a "buy" consensus rating for Apple stock, with an average rating score of "Aaa" based on 79 buy ratings, 14 hold ratings, and 4 sell ratings. The median price target from 97 analysts is $261.03, with a high estimate of $325.00 and a low of $173.00.

**Recent Developments & News:**

*   **Antitrust Challenge in India:** Apple is contesting India's antitrust penalty law in the Delhi High Court. The company argues that fines based on global revenue are unconstitutional, potentially risking penalties up to $38 billion.
*   **